Тестовое задание:
С использованием методов process mining написать алгоритм сбора статистики по логу и определить-
1. Количество всех возможных путей процесса 
2. Основной путь процесса 
3. Этапы, удлиняющие основной путь
4. Длительность этапов процесса

In [77]:
%pip install sberpm

Note: you may need to restart the kernel to use updated packages.


���⠪��᪠� �訡�� � ����� 䠩��, ����� ����� ��� ��⪥ ⮬�.


**Импорт необходимых зависимостей.**


In [78]:
from sberpm import DataHolder
import pandas as pd

In [79]:
path = 'static\log.csv'

In [80]:
df = pd.read_csv(path)

In [81]:
df['Дата'] = df.apply(lambda row: str(row['Дата']) + ' ' + str(row['Часовой пояс']), axis=1)
df

,Ид,Дата,Часовой пояс,Операция
0,10010.0,03.10.2022 2:16 UTC+0,UTC+0,Блокировка
1,10030.0,03.10.2022 2:20 UTC+0,UTC+0,Блокировка
2,10050.0,03.10.2022 4:20 UTC+0,UTC+0,Блокировка
3,10060.0,03.10.2022 4:50 UTC+0,UTC+0,Блокировка
4,10070.0,03.10.2022 5:01 UTC+0,UTC+0,Блокировка
...,...,...,...,...
67778,13295.0,12.10.2022 13:29 UTC+5,UTC+5,Регистрация
67779,13305.0,12.10.2022 13:42 UTC+5,UTC+5,Регистрация
67780,13315.0,12.10.2022 13:58 UTC+5,UTC+5,Регистрация
67781,13325.0,12.10.2022 14:15 UTC+5,UTC+5,Регистрация


In [82]:
data_holder = DataHolder(data=df, 
                         id_column='Ид', 
                         activity_column='Операция', 
                         start_timestamp_column='Дата',
                         text_column='Часовой пояс',
                         yearfirst= True)

In [83]:
data_holder.data.head()

,Ид,Дата,Часовой пояс,Операция
0,10001.0,2022-01-10 01:00:00+00:00,UTC+1,Прием взыскание
1,10001.0,2022-01-10 01:00:00+00:00,UTC+1,Прием взыскание
2,10001.0,2022-01-10 01:00:00+00:00,UTC+1,Прием взыскание
3,10001.0,2022-01-10 01:00:00+00:00,UTC+1,Прием взыскание
4,10001.0,2022-01-10 07:58:00+00:00,UTC+1,Регистрация


**Отрисовка всех ребер без фильтрации**

In [84]:
from sberpm.miners import SimpleMiner

In [85]:
simple_miner = SimpleMiner(data_holder)
simple_miner.apply()
graph = simple_miner.graph

In [86]:
graph

Nodes: ['Прием взыскание', 'Регистрация', 'Блокировка', 'Назначено в рабочую группу', 'Исполнено', 'Проверка_1', 'Проверка_2', 'Ожидание', 'Исполнeно', 'Прием арест', 'Проверка_3', 'startevent', 'endevent']
Edges: [('startevent', 'Прием взыскание'), ('startevent', 'Прием арест'), ('startevent', 'Исполнeно'), ('startevent', 'Исполнено'), ('startevent', 'Назначено в рабочую группу'), ('startevent', 'Проверка_2'), ('Исполнено', 'endevent'), ('Ожидание', 'endevent'), ('Исполнeно', 'endevent'), ('Назначено в рабочую группу', 'endevent'), ('Блокировка', 'endevent'), ('Проверка_1', 'endevent'), ('Исполнeно', 'Исполнeно'), ('Блокировка', 'Исполнeно'), ('Блокировка', 'Блокировка'), ('Прием взыскание', 'Проверка_1'), ('Исполнено', 'Исполнено'), ('Регистрация', 'Проверка_2'), ('Проверка_3', 'Проверка_3'), ('Исполнено', 'Прием взыскание'), ('Проверка_1', 'Назначено в рабочую группу'), ('Регистрация', 'Блокировка'), ('Проверка_2', 'Проверка_2'), ('Блокировка', 'Прием арест'), ('Проверка_1', 'Ожидан

In [87]:
%matplotlib inline
from sberpm.visual import GraphvizPainter


In [88]:
printer = GraphvizPainter()
printer.apply(graph)
printer.show()

In [89]:
from sberpm.miners import CausalMiner

In [90]:
causal_miner = CausalMiner(data_holder)
causal_miner.apply()
graph = causal_miner.graph

painter = GraphvizPainter()
painter.apply(graph)
painter.show()

**1. Колличество путей процесса**

In [91]:
from sberpm import metrics

In [92]:
trace_metric = metrics.TraceMetric(data_holder, time_unit='d')

In [93]:
all_ways_count = len(trace_metric.apply())
print(f'Колличество всех возможных путей в процессе: {all_ways_count}')

Колличество всех возможных путей в процессе: 49


In [94]:
all_ways = trace_metric.apply()

In [95]:
all_ways

,count,ids,trace_length,unique_activities_num,loop_percent,total_duration,mean_duration,median_duration,max_duration,min_duration,variance_duration,std_duration
"(Прием взыскание, Прием взыскание, Прием взыскание, Прием взыскание, Регистрация, Регистрация, Регистрация, Регистрация, Блокировка, Блокировка, Блокировка, Блокировка, Назначено в рабочую группу, Назначено в рабочую группу, Назначено в рабочую группу, Назначено в рабочую группу, Исполнено, Исполнено, Исполнено, Исполнено)",1016,"{10251.0, 10631.0, 11837.0, 12245.0, 10754.0, ...",20,5,75.000000,141773.082639,139.540436,153.047569,570.670139,0.405556,6.718487e+08,88.181779
"(Прием взыскание, Прием взыскание, Прием взыскание, Прием взыскание, Регистрация, Регистрация, Регистрация, Регистрация, Блокировка, Блокировка, Блокировка, Блокировка, Назначено в рабочую группу, Назначено в рабочую группу, Назначено в рабочую группу, Назначено в рабочую группу, Ожидание, Ожидание, Ожидание, Ожидание)",848,"{12603.0, 10155.0, 12941.0, 12826.0, 12546.0, ...",20,5,75.000000,442131.000000,521.380896,540.000000,721.000000,387.000000,8.794774e+08,100.891708
"(Прием взыскание, Прием взыскание, Прием взыскание, Прием взыскание, Регистрация, Регистрация, Регистрация, Регистрация, Блокировка, Блокировка, Блокировка, Блокировка, Назначено в рабочую группу, Назначено в рабочую группу, Назначено в рабочую группу, Назначено в рабочую группу, Исполнeно, Исполнeно, Исполнeно, Исполнeно)",499,"{11337.0, 11546.0, 12306.0, 10464.0, 12197.0, ...",20,5,75.000000,65449.461806,131.161246,129.423611,320.470139,0.485417,5.725759e+08,81.406608
"(Исполнено, Исполнено, Исполнено, Исполнено, Прием взыскание, Прием взыскание, Прием взыскание, Прием взыскание, Регистрация, Регистрация, Регистрация, Регистрация, Блокировка, Блокировка, Блокировка, Блокировка, Назначено в рабочую группу, Назначено в рабочую группу, Назначено в рабочую группу, Назначено в рабочую группу)",336,"{12607.0, 12575.0, 12890.0, 12613.0, 12623.0, ...",20,5,75.000000,14692.938889,43.728985,22.835764,302.026389,12.300694,1.764392e+08,45.189828
"(Исполнeно, Исполнeно, Исполнeно, Исполнeно, Прием взыскание, Прием взыскание, Прием взыскание, Прием взыскание, Регистрация, Регистрация, Регистрация, Регистрация, Блокировка, Блокировка, Блокировка, Блокировка, Назначено в рабочую группу, Назначено в рабочую группу, Назначено в рабочую группу, Назначено в рабочую группу)",212,"{13168.0, 12892.0, 13211.0, 13208.0, 12529.0, ...",20,5,75.000000,10339.208333,48.769851,49.936806,302.009028,15.877083,1.124433e+08,36.075298
"(Прием взыскание, Прием взыскание, Прием взыскание, Прием взыскание, Регистрация, Регистрация, Регистрация, Регистрация, Назначено в рабочую группу, Назначено в рабочую группу, Назначено в рабочую группу, Назначено в рабочую группу, Блокировка, Блокировка, Блокировка, Блокировка, Исполнено, Исполнено, Исполнено, Исполнено)",62,"{10153.0, 10354.0, 10454.0, 11643.0, 11934.0, ...",20,5,75.000000,8956.562500,144.460685,127.932639,275.639583,3.045833,6.929404e+08,89.555254
"(Прием взыскание, Прием взыскание, Прием взыскание, Прием взыскание, Регистрация, Регистрация, Регистрация, Регистрация, Назначено в рабочую группу, Назначено в рабочую группу, Назначено в рабочую группу, Назначено в рабочую группу, Блокировка, Блокировка, Блокировка, Блокировка, Ожидание, Ожидание, Ожидание, Ожидание)",56,"{13210.0, 13120.0, 11236.0, 11073.0, 13305.0, ...",20,5,75.000000,28904.000000,516.142857,524.500000,662.000000,387.000000,9.336613e+08,103.953188
"(Прием взыскание, Прием взыскание, Прием взыскание, Прием взыскание, Регистрация, Регистрация, Регистрация, Регистрация, Проверка_1, Проверка_1, Проверка_1, Проверка_1, Проверка_2, Проверка_2, Проверка_2, Проверка_2, Назначено в рабочую группу, Назначено в рабочую группу, Назначено в рабочую группу, Назначено в рабочую группу, Блокировка, Блокировка, Блокировка, Блокировка, Ожидание, Ожидание, Ожидание, Ожидание)",52,"{13065.0, 12526.0, 11216.0, 11113.0, 11013.0, ...",28,7,75.000000,27600.000000,530.76923

**2. Основной путь процесса**

Осмелюсь предположить что основной путь процесса скрывается за цепочкой с самым длинным исполнением.

In [96]:
from sberpm.metrics import TransitionMetric 

все узлы процесса (Для casual без петлей)

In [97]:
graph.get_nodes()

[Node: Прием взыскание,
 Node: Регистрация,
 Node: Блокировка,
 Node: Назначено в рабочую группу,
 Node: Исполнено,
 Node: Проверка_1,
 Node: Проверка_2,
 Node: Ожидание,
 Node: Исполнeно,
 Node: Прием арест,
 Node: Проверка_3,
 Node: startevent,
 Node: endevent]

Длительность выполнения каждого шага

In [98]:
from sberpm.metrics import ActivityMetric

In [99]:
activity_metric = ActivityMetric(data_holder, time_unit='d')
activity_metric.max_duration()

Операция
Блокировка                    661.822917
Исполнeно                     332.454861
Исполнено                     301.704167
Назначено в рабочую группу    661.906944
Ожидание                        0.000000
Прием арест                     0.646528
Прием взыскание                 0.978472
Проверка_1                    386.456944
Проверка_2                     91.817361
Проверка_3                    152.707639
Регистрация                    60.963194
Name: max_duration, dtype: float64

Ищем цепочку с максимально долгой продолжительностью

In [100]:
from sberpm.metrics import TraceMetric

Длительность самой продолжительной цепочки


In [101]:
trace_metric = TraceMetric(data_holder, time_unit='d')

total_d = trace_metric.total_duration()

max(total_d)

442131.0

In [102]:
main_way_id = pd.Series.idxmax(total_d)

In [103]:
main_way_id

('Прием взыскание',
 'Прием взыскание',
 'Прием взыскание',
 'Прием взыскание',
 'Регистрация',
 'Регистрация',
 'Регистрация',
 'Регистрация',
 'Блокировка',
 'Блокировка',
 'Блокировка',
 'Блокировка',
 'Назначено в рабочую группу',
 'Назначено в рабочую группу',
 'Назначено в рабочую группу',
 'Назначено в рабочую группу',
 'Ожидание',
 'Ожидание',
 'Ожидание',
 'Ожидание')

Цепочка

In [104]:
total_d = total_d.to_dict()
max(total_d)

('Проверка_2',
 'Проверка_2',
 'Проверка_2',
 'Проверка_2',
 'Назначено в рабочую группу',
 'Назначено в рабочую группу',
 'Назначено в рабочую группу',
 'Назначено в рабочую группу',
 'Блокировка',
 'Блокировка',
 'Блокировка',
 'Блокировка',
 'Прием взыскание',
 'Прием взыскание',
 'Прием взыскание',
 'Прием взыскание',
 'Регистрация',
 'Регистрация',
 'Регистрация',
 'Регистрация',
 'Проверка_1',
 'Проверка_1',
 'Проверка_1',
 'Проверка_1',
 'Ожидание',
 'Ожидание',
 'Ожидание',
 'Ожидание')

**3. Этапы, удлиняющие основной путь**

In [107]:
activities = trace_metric.unique_activities()
activities[main_way_id]

{'Блокировка',
 'Назначено в рабочую группу',
 'Ожидание',
 'Прием взыскание',
 'Регистрация'}

Смею предположить что увеличение времени основного процесса кроется в операциях 'Блокировка', 'Отправка в общую группу'. А так же возникают вопросы к петлям внутр графа. Возможно это возникает по причине того что я неправильно использовал поля часовой пояс.


**4. Длительность этапов процесса**

In [106]:
transition_metric = TransitionMetric(data_holder, time_unit='d')
transition_metric.total_duration()

transition
(Блокировка, Блокировка)                                         0.000000
(Блокировка, Исполнeно)                                       8395.687500
(Блокировка, Исполнено)                                      15082.461806
(Блокировка, Назначено в рабочую группу)                         0.000000
(Блокировка, Ожидание)                                       62543.340278
(Блокировка, Прием арест)                                       56.350694
(Блокировка, Прием взыскание)                                   57.593056
(Исполнeно, Исполнeно)                                           0.000000
(Исполнeно, Прием взыскание)                                 11792.913194
(Исполнено, Исполнено)                                           0.000000
(Исполнено, Прием взыскание)                                 16000.336806
(Назначено в рабочую группу, Блокировка)                         2.761806
(Назначено в рабочую группу, Исполнeно)                      68059.359722
(Назначено в рабочую группу